In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/MyDrive/spam_or_not_spam.csv')

In [3]:
df.head(10)

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0
5,i just had to jump in here as carbonara is on...,0
6,the scotsman NUMBER august NUMBER playboy want...,0
7,martin adamson wrote isn t it just basically a...,0
8,the scotsman thu NUMBER aug NUMBER meaningful ...,0
9,i have been trying to research via sa mirrors ...,0


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [7]:
df.dropna(subset=['email'], inplace=True)

vectorizer = TfidfVectorizer(max_features=1000)

X = vectorizer.fit_transform(df['email']).toarray()

y = df['label'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train_ham = X_train[y_train == 0]

input_dim = X_train_ham.shape[1]

In [10]:
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
output_layer = Dense(input_dim, activation='sigmoid')(decoded)

In [11]:
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [34]:
autoencoder.fit(X_train_ham, X_train_ham, epochs=35, batch_size=32, validation_split=0.2)

Epoch 1/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0222 - val_loss: 0.0362
Epoch 2/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0226 - val_loss: 0.0361
Epoch 3/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0223 - val_loss: 0.0362
Epoch 4/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0226 - val_loss: 0.0362
Epoch 5/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0221 - val_loss: 0.0362
Epoch 6/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0223 - val_loss: 0.0363
Epoch 7/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0229 - val_loss: 0.0364
Epoch 8/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0226 - val_loss: 0.0363
Epoch 9/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0227 - val_loss: 0.0362
Epoch 10/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0225 - val_loss: 0.0364
Epoch 11/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0229 - val_loss: 0.0364
Epoch 12/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0224 - val_lo

In [35]:
reconstructed_train = autoencoder.predict(X_train_ham)
reconstruction_error_train = np.mean(np.abs(X_train_ham - reconstructed_train), axis=1)

ham_mask = np.ones_like(reconstruction_error_train, dtype=bool)

threshold = np.percentile(reconstruction_error_train[ham_mask], 95)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [36]:
y_pred = (reconstruction_error > threshold).astype(int)

In [38]:
y_pred

array([0, 0, 0, ..., 0, 1, 0])

In [37]:
print("Accuracy:", accuracy_score(y, y_pred))
print("Classification Report:\n", classification_report(y, y_pred))

Accuracy: 0.8032677559186395
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.83      0.88      2500
           1       0.44      0.66      0.53       499

    accuracy                           0.80      2999
   macro avg       0.68      0.74      0.70      2999
weighted avg       0.84      0.80      0.82      2999



In [29]:
autoencoder.save('/content/drive/MyDrive/email_autoencoder.h5')